In [1]:
import torch
from torch.autograd import Variable as var
from torch import nn, cuda
from torch.utils.data import Dataset, DataLoader, sampler
import talib as ta
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import pandas as pd
from urllib import request
import requests
from time import sleep
import chardet
import sys
from pprint import pprint
import itertools

def get_file_list(root, ftype = ".csv"):
    import os
    import sys
    FileList = []
    filename = []
    for dirPath, dirNames, fileNames in os.walk(root):
        for f in fileNames:
            if f.find(ftype) > -1:
                FileList.append(os.path.join(dirPath, f))
                filename.append(f.replace(ftype, ""))
    if len(filename) > 0:
        a = zip(FileList, filename)
        a = sorted(a, key = lambda t : t[1])
        FileList, filename = zip(*a)
    return FileList, filename

In [2]:
etf18 = pd.read_csv("../TBrain/DataSet/tetfp.csv", encoding="big5hkscs", low_memory=False)
etf18 = etf18.rename(columns={'中文簡稱':'證券名稱'})
etf18 = etf18.dropna(axis=1, how='all')
etf18 = etf18.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
etf18 = etf18.drop_duplicates()
etf18 = etf18.reset_index()
date_list = list(np.unique(etf18["日期"].values))
code_uq = list(etf18["代碼"].unique())
finalETF18 = pd.read_csv("../TBrain/DataSet/final18_"+str(date_list[-1])+".csv")


In [3]:
score_detail = {
    "./20180615/l2_result_20180615.csv": {
        "avg": [
            0.2500000, 0.4967936,
            0.0750000, 0.4953169,
            0.1500000,0.4965883,
            0.1500000,0.4972381,
            0.1500000,0.4925524,
            0.2500000,0.4941123,
            0.2000000,0.4955399,
            0.1750000,0.4951611,
            0.1500000,0.4934693,
            0.2750000,0.4955464,
            0.1500000,0.4984418,
            0.1000000,0.4951516,
            0.2750000,0.4939040,
            0.1000000,0.4956571,
            0.2250000,0.4951340,
            0.2250000,0.4955980,
            0.2500000,0.4962492,
            0.1500000,0.4960020,
        ],
        "avg2": [
            0.1750000,0.4909689,
            0.2250000,0.4907654,
            0.2750000,0.4957659,
            0.0500000,0.4894872,
            0.2250000,0.4837602,
            0.2500000,0.4901427,
            0.1500000,0.4947093,
            0.2750000,0.4904451,
            0.5000000,0.4878140,
            0.3750000,0.4903258,
            0.2250000,0.4941418,
            0.2250000,0.4892786,
            0.2250000,0.4901852,
            0.2500000,0.4885035,
            0.2000000,0.4918794,
            0.3500000,0.4968957,
            0.3500000,0.4927437,
            0.1500000,0.4940803,
        ]
    },
    "./20180615/l1_result_20180615.csv": {
        "avg": [
            0.3000000,0.4967936,
            0.0750000,0.4953169,
            0.1500000,0.4965883,
            0.3500000,0.4972381,
            0.4000000,0.4925524,
            0.2500000,0.4941123,
            0.0500000,0.4955399,
            0.1000000,0.4951611,
            0.0000000,0.4934693,
            0.0000000,0.4955464,
            0.1250000,0.4984418,
            0.1000000,0.4951516,
            0.2750000,0.4939040,
            0.2750000,0.4956571,
            0.3000000,0.4951340,
            0.0000000,0.4955980,
            0.2500000,0.4962492,
            0.1500000,0.4960020,
        ], 
        "avg2": [
            0.2750000,0.4909689,
            0.0500000,0.4907654,
            0.2750000,0.4957659,
            0.2500000,0.4894872,
            0.3500000,0.4837602,
            0.1250000,0.4901427,
            0.1500000,0.4947093,
            0.3500000,0.4904451,
            0.3500000,0.4878140,
            0.3500000,0.4903258,
            0.1500000,0.4941418,
            0.3500000,0.4892786,
            0.3500000,0.4901852,
            0.3500000,0.4885035,
            0.2750000,0.4918794,
            0.1500000,0.4968957,
            0.3500000,0.4927437,
            0.2250000,0.4940803,
        ]
    },
    "./20180615/cr_result_20180615.csv": {
        "avg": [
            0.0000000,0.4934637,
            0.1500000,0.4656866,
            0.2000000,0.4771325,
            0.1500000,0.4677011,
            0.1500000,0.4619415,
            0.0000000,0.4535467,
            0.3000000,0.4767051,
            0.1500000,0.4614220,
            0.2500000,0.4646342,
            0.1000000,0.4818235,
            0.3750000,0.4741587,
            0.1500000,0.4688864,
            0.0000000,0.4779700,
            0.1500000,0.4771979,
            0.2000000,0.4621930,
            0.1000000,0.4854483,
            0.0000000,0.4659669,
            0.2000000,0.4617158,
        ], 
        "avg2": [
            0.1000000,0.4963061,
            0.1250000,0.4934774,
            0.3250000,0.4492032,
            0.4250000,0.4803177,
            0.4250000,0.4912034,
            0.3000000,0.4899327,
            0.0750000,0.4729981,
            0.5000000,0.4776895,
            0.3000000,0.4874251,
            0.4000000,0.4945078,
            0.2250000,0.4829229,
            0.3250000,0.4806800,
            0.2750000,0.4936907,
            0.2000000,0.4778842,
            0.3750000,0.4852703,
            0.1250000,0.4922449,
            0.3000000,0.4965763,
            0.4250000,0.4946149,
        ]
    },
    "./20180615/cr2_result_20180615.csv": {
        "avg": [
            0.1500000,0.4883237,
            0.0750000,0.4594319,
            0.1250000,0.4766685,
            0.0750000,0.4591299,
            0.0750000,0.4432176,
            0.3500000,0.4685540,
            0.3500000,0.4590600,
            0.0750000,0.4530949,
            0.1750000,0.4609085,
            0.1750000,0.4654624,
            0.3000000,0.4732158,
            0.0750000,0.4673572,
            0.2250000,0.4600603,
            0.0750000,0.4662399,
            0.4000000,0.4556741,
            0.0000000,0.4687379,
            0.1500000,0.4679612,
            0.1250000,0.4639196,
        ], 
        "avg2": [
            0.0500000,0.4903738,
            0.3750000,0.4902582,
            0.3750000,0.4882622,
            0.3250000,0.4870517,
            0.1250000,0.4823071,
            0.1500000,0.4799623,
            0.3250000,0.4864435,
            0.2250000,0.4877057,
            0.4000000,0.4818369,
            0.3500000,0.4836097,
            0.0500000,0.4902525,
            0.3250000,0.4827908,
            0.1250000,0.4926693,
            0.4250000,0.4936644,
            0.0500000,0.4911187,
            0.4000000,0.4938224,
            0.3500000,0.4861772,
            0.4250000,0.4956653,
        ]
    },
    "./20180615/j5_result_20180615.csv": {
        "avg": [
            0.3250000,0.4962397,
            0.0500000,0.4956763,
            0.2000000,0.4926035,
            0.0750000,0.4942256,
            0.1750000,0.4942960,
            0.1250000,0.4820990,
            0.0500000,0.4922762,
            0.2000000,0.4956933,
            0.0000000,0.4978791,
            0.0000000,0.4981897,
            0.2750000,0.4818516,
            0.2250000,0.4967965,
            0.1000000,0.4975245,
            0.2750000,0.4969820,
            0.2500000,0.4911853,
            0.1250000,0.4938681,
            0.2000000,0.4933525,
            0.1500000,0.4970511,
        ]
    },
    "./20180615/j15_result_20180615.csv": {
        "avg": [
            0.3000000,0.4969345,
            0.2000000,0.4950284,
            0.2750000,0.4926428,
            0.2750000,0.4926022,
            0.1750000,0.4934938,
            0.2500000,0.4815625,
            0.0500000,0.4935438,
            0.1000000,0.4952264,
            0.2250000,0.4970503,
            0.0000000,0.4980218,
            0.5000000,0.4812252,
            0.1750000,0.4958972,
            0.2500000,0.4979342,
            0.2500000,0.4978527,
            0.2750000,0.4931639,
            0.0000000,0.4947653,
            0.2250000,0.4934257,
            0.1500000,0.4971617,
        ], 
        "avg2": [
            0.2250000,0.4876939,
            0.4500000,0.4840256,
            0.1250000,0.4966384,
            0.4500000,0.4820677,
            0.4500000,0.4788970,
            0.4250000,0.4935973,
            0.1500000,0.4898358,
            0.3500000,0.4975038,
            0.5000000,0.4934349,
            0.2250000,0.4850044,
            0.1250000,0.4947098,
            0.4500000,0.4912283,
            0.1750000,0.4878040,
            0.4000000,0.4936197,
            0.3500000,0.4883819,
            0.2500000,0.4970616,
            0.2000000,0.4925848,
            0.4000000,0.4944033,
        ]
    },
    "./20180615/M_result_20180615.csv": {
        "avg": [
            0.2250000,0.4969918,
            0.3500000,0.4960071,
            0.2250000,0.4957700,
            0.3750000,0.4977715,
            0.1000000,0.4902354,
            0.4000000,0.4938947,
            0.0000000,0.4936764,
            0.1500000,0.4806224,
            0.2500000,0.4777554,
            0.3000000,0.4935524,
            0.2750000,0.4981129,
            0.3250000,0.4865596,
            0.3750000,0.4890449,
            0.0750000,0.4877502,
            0.1500000,0.4972350,
            0.1000000,0.4866988,
            0.4250000,0.4943458,
            0.3000000,0.4972115,
        ],
        "avg2": [
            0.3500000,0.4938353,
            0.2750000,0.4928769,
            0.3500000,0.4951644,
            0.3500000,0.4916747,
            0.3500000,0.4869288,
            0.1750000,0.4919532,
            0.2250000,0.4968123,
            0.2750000,0.4919621,
            0.4250000,0.4897193,
            0.2750000,0.4925473,
            0.2250000,0.4935923,
            0.4250000,0.4917914,
            0.2750000,0.4916878,
            0.2750000,0.4914916,
            0.3500000,0.4937692,
            0.3500000,0.4968618,
            0.2750000,0.4947505,
            0.1500000,0.4964524,
        ]
    },
    "./20180615/aa_result_20180615.csv": {
        "avg": [
            0.1500000,0.4569847,
            0.1250000,0.4957321,
            0.0500000,0.4931150,
            0.1250000,0.4969030,
            0.3500000,0.4933472,
            0.0500000,0.4961521,
            0.2000000,0.3856442,
            0.4500000,0.4981976,
            0.1500000,0.4917837,
            0.2250000,0.4966587,
            0.3250000,0.4774137,
            0.1250000,0.4961500,
            0.1000000,0.4921722,
            0.1250000,0.4976174,
            0.3000000,0.4876439,
            0.1250000,0.4945048,
            0.3750000,0.4932321,
            0.3250000,0.4953116,
        ],
        "avg2": [
            0.4500000,0.4929396,
            0.4000000,0.4888649,
            0.2500000,0.4860128,
            0.3250000,0.4916436,
            0.2250000,0.4891936,
            0.2250000,0.4483731,
            0.0500000,0.4890173,
            0.3750000,0.4959038,
            0.4000000,0.4812123,
            0.2500000,0.4597251,
            0.2750000,0.4566901,
            0.1750000,0.4892285,
            0.3000000,0.4911781,
            0.1750000,0.4925726,
            0.2750000,0.4962956,
            0.4000000,0.4876112,
            0.2750000,0.4509031,
            0.0500000,0.4913614,
        ]
    },
    "./20180615/a3_result_20180615.csv": {
        "avg": [
            0.3000000,0.4968842,
            0.0000000,0.4950741,
            0.1500000,0.4969396,
            0.1000000,0.4964991,
            0.1000000,0.4909545,
            0.2500000,0.4944734,
            0.0500000,0.4953198,
            0.1000000,0.4959874,
            0.0000000,0.4942095,
            0.0000000,0.4961317,
            0.1250000,0.4982668,
            0.1000000,0.4955589,
            0.2500000,0.4942215,
            0.1000000,0.4960112,
            0.3000000,0.4953900,
            0.0000000,0.4965495,
            0.2500000,0.4971683,
            0.1500000,0.4956841,
        ],
        "avg2": [
            0.2750000,0.4944714,
            0.3500000,0.4979869,
            0.2750000,0.4967665,
            0.2750000,0.4964803,
            0.2750000,0.4961271,
            0.2500000,0.4923400,
            0.1500000,0.4934432,
            0.3500000,0.4956128,
            0.3500000,0.4939906,
            0.3500000,0.4972047,
            0.1500000,0.4823769,
            0.3500000,0.4962512,
            0.3500000,0.4950288,
            0.3500000,0.4942458,
            0.2750000,0.4943525,
            0.2750000,0.4901133,
            0.3500000,0.4929099,
            0.2250000,0.4955446,
        ]
    }
}


In [4]:
files = sorted(list(score_detail.keys()), key=str.lower)

tag = [x.replace("./20180615/", "").replace("_result_20180615.csv", "") for x in files]
files


['./20180615/a3_result_20180615.csv',
 './20180615/aa_result_20180615.csv',
 './20180615/cr2_result_20180615.csv',
 './20180615/cr_result_20180615.csv',
 './20180615/j15_result_20180615.csv',
 './20180615/j5_result_20180615.csv',
 './20180615/l1_result_20180615.csv',
 './20180615/l2_result_20180615.csv',
 './20180615/M_result_20180615.csv']

In [5]:
for f in files:
    if "avg2" in score_detail[f]:
        score_detail[f]["avg"] = np.array(score_detail[f]["avg"]) * 1.618 + np.array(score_detail[f]["avg2"])
    else:
        score_detail[f]["avg"] = np.array(score_detail[f]["avg"])
    

In [6]:
avg_to_df = []
for x in files:
    avg_to_df.append(score_detail[x]['avg'])
col_label = []
for c in code_uq:
    for x in ["ud","cprice"]:
        col_label.append("{}_{}".format(c, x))
df_score = pd.DataFrame(np.array(avg_to_df).T, columns=tag)
score2 = df_score.values
df_score.insert(0, 'code\Model', col_label)
df_score.to_csv("./en_valid.csv", index=False)
df_score


,code\Model,a3,aa,cr2,cr,j15,j5,l1,l2,M
0,50_ud,0.760400,0.692700,0.292700,0.100000,0.710400,0.325000,0.760400,0.579500,0.714050
1,50_cprice,1.298430,1.232341,1.280482,1.294730,1.291734,0.496240,1.294781,1.294781,1.297968
2,51_ud,0.350000,0.602250,0.496350,0.367700,0.773600,0.050000,0.171350,0.346350,0.841300
3,51_cprice,1.299017,1.290959,1.233619,1.246958,1.284982,0.495676,1.292188,1.292188,1.295416
4,52_ud,0.517700,0.330900,0.577250,0.648600,0.569950,0.200000,0.517700,0.517700,0.714050
5,52_cprice,1.300815,1.283873,1.259512,1.221204,1.293734,0.492604,1.299246,1.299246,1.297320
6,53_ud,0.436800,0.527250,0.446350,0.667700,0.894950,0.075000,0.816300,0.292700,0.956750
7,53_cprice,1.299816,1.295633,1.229924,1.237058,1.279098,0.494226,1.294018,1.294018,1.297069
8,54_ud,0.436800,0.791300,0.246350,0.667700,0.733150,0.175000,0.997200,0.467700,0.511800
9,54_cprice,1.290491,1.287429,1.199433,1.238625,1.277370,0.494296,1.280710,1.280710,1.280130


In [7]:
tot = np.zeros(36)
for x in tag:
    tot += df_score[x].values
print(tot)
weight = {}
for x in files:
    tmp = []
    for i_c, c in enumerate(code_uq):
        tmp2 = []
        for d in range(5):
            tmp2.append(score_detail[x]['avg'][i_c * 2] / tot[i_c * 2])
            tmp2.append(score_detail[x]['avg'][i_c * 2 + 1] / tot[i_c * 2 + 1])
        tmp.append(np.array(tmp2))
    weight.update({x:np.array(tmp)})
    

[  4.93515     10.78148634   3.9989      10.73100409   4.59385     10.74755281
   5.1138      10.72085999   5.027       10.62919439   4.9374      10.63057114
   3.2666      10.55060208   5.0034      10.69250555   5.1666      10.66809356
   4.31435     10.73843896   5.21915     10.6797094    4.7107      10.71867673
   5.0065      10.73887716   4.5607      10.7502926    5.8787      10.70450916
   3.3149      10.78453787   5.76465     10.71038191   4.7079      10.76832103]


In [8]:
weight


{'./20180615/M_result_20180615.csv': array([[ 0.14468659,  0.1203886 ,  0.14468659,  0.1203886 ,  0.14468659,
          0.1203886 ,  0.14468659,  0.1203886 ,  0.14468659,  0.1203886 ],
        [ 0.21038286,  0.12071716,  0.21038286,  0.12071716,  0.21038286,
          0.12071716,  0.21038286,  0.12071716,  0.21038286,  0.12071716],
        [ 0.15543607,  0.12070843,  0.15543607,  0.12070843,  0.15543607,
          0.12070843,  0.15543607,  0.12070843,  0.15543607,  0.12070843],
        [ 0.18709179,  0.12098554,  0.18709179,  0.12098554,  0.18709179,
          0.12098554,  0.18709179,  0.12098554,  0.18709179,  0.12098554],
        [ 0.10181022,  0.12043525,  0.10181022,  0.12043525,  0.10181022,
          0.12043525,  0.10181022,  0.12043525,  0.10181022,  0.12043525],
        [ 0.16652489,  0.12144924,  0.16652489,  0.12144924,  0.16652489,
          0.12144924,  0.16652489,  0.12144924,  0.16652489,  0.12144924],
        [ 0.06887896,  0.12279685,  0.06887896,  0.12279685,  0.068878

In [9]:
fin = np.zeros(180).reshape(18, 10)
for x in files:
    a = pd.read_csv(x)
    d = a.pop('ETFid')
    a = a.values
    fin += np.multiply(weight[x], a)
    

In [10]:
fin_df = pd.DataFrame(fin, columns=["Mon_ud","Mon_cprice","Tue_ud","Tue_cprice", "Wed_ud","Wed_cprice","Thu_ud","Thu_cprice","Fri_ud","Fri_cprice"])
for x in fin_df.columns:
    if x.find("ud") > -1:
        fin_df[x][fin_df[x] > 0] = 1
        fin_df[x][fin_df[x] < 0] = -1
fin_df.insert(0, 'ETFid', code_uq)


In [11]:
fin_df.to_csv("majority_"+str(date_list[-1])+".csv", index=False)
fin_df


,ETFid,Mon_ud,Mon_cprice,Tue_ud,Tue_cprice,Wed_ud,Wed_cprice,Thu_ud,Thu_cprice,Fri_ud,Fri_cprice
0,50,1.0,81.512688,-1.0,81.384629,-1.0,80.487506,-1.0,80.395531,1.0,81.919502
1,51,1.0,34.246118,-1.0,34.190332,1.0,34.375562,1.0,34.317453,-1.0,34.266547
2,52,1.0,51.351497,-1.0,51.382536,1.0,51.411147,-1.0,51.346869,-1.0,51.494855
3,53,1.0,36.330214,1.0,36.580684,1.0,36.466806,-1.0,36.331722,1.0,36.606340
4,54,1.0,24.601876,1.0,24.684379,1.0,24.859062,1.0,25.030184,1.0,25.026092
5,55,1.0,17.503564,1.0,17.597700,1.0,17.611386,1.0,17.695292,-1.0,17.664351
6,56,1.0,26.330649,1.0,26.504166,1.0,26.606696,-1.0,26.548249,-1.0,26.455200
7,57,1.0,50.090856,-1.0,49.770110,1.0,50.023334,1.0,50.367923,-1.0,50.269425
8,58,1.0,47.795564,1.0,47.762552,1.0,48.318806,-1.0,48.286441,1.0,48.250151
9,59,-1.0,42.805226,1.0,42.931307,1.0,42.944606,-1.0,43.209882,-1.0,42.965864
